# 1. Activate GPU and Install Dependencies

#2. Preprocess data

In [1]:
# Load data
from datasets import load_dataset
data = load_dataset("sst2")

Using custom data configuration default
Found cached dataset sst2 (/zhome/a6/6/127219/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# Create a smaller training dataset for faster training times
small_train_dataset = data["train"].shuffle(seed=42).select([i for i in list(range(100))])
small_test_dataset = data["validation"].shuffle(seed=42).select([i for i in list(range(50))])
print(small_train_dataset[0])
print(small_test_dataset[0])

Loading cached shuffled indices for dataset at /zhome/a6/6/127219/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-da59dc4ea8815b88.arrow
Loading cached shuffled indices for dataset at /zhome/a6/6/127219/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-4300dba8468dfa9f.arrow


{'idx': 32326, 'sentence': 'klein , charming in comedies like american pie and dead-on in election , ', 'label': 1}
{'idx': 726, 'sentence': 'it gets onto the screen just about as much of the novella as one could reasonably expect , and is engrossing and moving in its own right . ', 'label': 1}


In [11]:
# Set DistilBERT tokenizer
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

loading file vocab.json from cache at /zhome/a6/6/127219/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
loading file merges.txt from cache at /zhome/a6/6/127219/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /zhome/a6/6/127219/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "i

In [12]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [14]:
# Define DistilBERT as our base model:
from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

loading configuration file config.json from cache at /zhome/a6/6/127219/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /zhome/a6/6/127219/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin

In [15]:
# Define the evaluation metrics 
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [22]:
# Log in to your Hugging Face account 
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

In [27]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

#repo_name = "/work3/s174498/finetuning-sentiment-model-3000-samples"
repo_name = "/work3/s174498/finetuning-sentiment-model-300-samples"
training_args = TrainingArguments(
    output_dir=repo_name,
    overwrite_output_dir = 'True',
    evaluation_strategy="steps",
    eval_steps=40,
    logging_steps = 40,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    seed=0,
    lr_scheduler_type = "linear",
    weight_decay=0.01,
    save_strategy = "no",
    push_to_hub=False,
    save_total_limit = 2,
    load_best_model_at_end=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
# Train the model
trainer.train()
trainer.save_model(repo_name)

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/zhome/a6/6/127219/miniconda3/envs/sent_env/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 100
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 14


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /work3/s174498/finetuning-sentiment-model-300-samples
Configuration saved in /work3/s174498/finetuning-sentiment-model-300-samples/config.json
Model weights saved in /work3/s174498/finetuning-sentiment-model-300-samples/pytorch_model.bin
tokenizer config file saved in /work3/s174498/finetuning-sentiment-model-300-samples/tokenizer_config.json
Special tokens file saved in /work3/s174498/finetuning-sentiment-model-300-samples/special_tokens_map.json


In [18]:
# Compute the evaluation metrics
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 16


/zhome/a6/6/127219/miniconda3/envs/sent_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


{'eval_loss': 0.6874444484710693,
 'eval_accuracy': 0.58,
 'eval_f1': 0.7341772151898733,
 'eval_runtime': 8.1002,
 'eval_samples_per_second': 6.173,
 'eval_steps_per_second': 0.494,
 'epoch': 2.0}

# 4. Analyzing new data with the model

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="federicopascual/finetuning-sentiment-model-3000-samples")

sentiment_model(["I love this move", "This movie sucks!"])

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

loading configuration file https://huggingface.co/federicopascual/finetuning-sentiment-model-3000-samples/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6acaf454bd6ace68f425fac5d8bbfc119633e7e1f8f653b4e828a89a9ef13e75.7a44e794a6ac10a8d9a844cfcb5e74a5d94ae645ec8273f74ab0f0784099bc9b
Model config DistilBertConfig {
  "_name_or_path": "federicopascual/finetuning-sentiment-model-3000-samples",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "vocab_size": 30522
}

loading c

[{'label': 'LABEL_1', 'score': 0.9558863043785095},
 {'label': 'LABEL_0', 'score': 0.9413502216339111}]